In [1]:
import pandas as pd
import numpy as np
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import (IntegerType,
 StringType,DecimalType,StructType,StructField,
 ArrayType,DoubleType,FloatType)
import pyspark.sql.functions as func
from pyspark.ml.feature import (StringIndexer, 
                                OneHotEncoder,
                                OneHotEncoderEstimator, 
                                VectorAssembler)
from pyspark.ml.classification import (LogisticRegression, 
                                       LogisticRegressionModel, 
                                       DecisionTreeClassifier)
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as evals
from pyspark.ml.evaluation import Evaluator
import pyspark.ml.tuning as tune
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

sc = SparkContext('local[*]')
spark = SparkSession(sc)

In [52]:
from pyspark.ml.classification import (RandomForestClassifier)

In [17]:
class BinaryLogLossEvaluator(Evaluator):
    def __init__(self,predictionCol='probability',labelCol='label'):
        self.predictionCol = predictionCol
        self.labelCol = labelCol

    def _get_probabilities(self,row):
        return row[1].item()
        
    def _evaluate(self,dataset):
        _udf_get_probabilities = func.udf(self._get_probabilities,FloatType())
        dataset = dataset.withColumn('prob_1',_udf_get_probabilities(dataset.select(self.predictionCol)[0]))
        
        self.log_loss = (-1) * dataset.select(func.sum(dataset.select(self.labelCol)[0] 
                                                           * func.log(dataset.prob_1)
                                                       + (1 - dataset.select(self.labelCol)[0])
                                                           * func.log(1-dataset.prob_1)
                                                      )
                                             ).collect()[0][0]
        
        return self.log_loss
    
    def isLargerBetter(self):
        return False

In [2]:
data_path = '../data/raw/shots_2007-2018.csv'

sample = pd.read_csv(data_path,nrows=10)
column_names = sample.columns

schema = StructType([StructField(x,StringType(),True) for x in column_names])

shots = spark.read.csv(data_path,schema=schema,
                       enforceSchema=True,header=True,ignoreLeadingWhiteSpace=True,
                       ignoreTrailingWhiteSpace=True)

for ix,x in enumerate(column_names):
    print(ix,x)

0 shotID
1 homeTeamCode
2 awayTeamCode
3 season
4 isPlayoffGame
5 game_id
6 homeTeamWon
7 id
8 time
9 timeUntilNextEvent
10 timeSinceLastEvent
11 period
12 team
13 location
14 event
15 goal
16 shotPlayContinuedOutsideZone
17 shotPlayContinuedInZone
18 shotGoalieFroze
19 shotPlayStopped
20 shotGeneratedRebound
21 homeTeamGoals
22 awayTeamGoals
23 xCord
24 yCord
25 xCordAdjusted
26 yCordAdjusted
27 shotAngle
28 shotAngleAdjusted
29 shotAnglePlusRebound
30 shotAngleReboundRoyalRoad
31 shotDistance
32 shotType
33 shotOnEmptyNet
34 shotRebound
35 shotAnglePlusReboundSpeed
36 shotRush
37 speedFromLastEvent
38 lastEventxCord
39 lastEventyCord
40 distanceFromLastEvent
41 lastEventShotAngle
42 lastEventShotDistance
43 lastEventCategory
44 lastEventTeam
45 homeEmptyNet
46 awayEmptyNet
47 homeSkatersOnIce
48 awaySkatersOnIce
49 awayPenalty1TimeLeft
50 awayPenalty1Length
51 homePenalty1TimeLeft
52 homePenalty1Length
53 playerPositionThatDidEvent
54 playerNumThatDidEvent
55 playerNumThatDidLastEven

In [3]:
shots = shots.withColumn('skaterDifference',shots.homeSkatersOnIce - shots.awaySkatersOnIce)
features_list = ['arenaAdjustedShotDistance','shotAngleAdjusted','shotRush','skaterDifference',
                 'shotOnEmptyNet','shotRebound','offWing','shooterTimeOnIce',
                 'averageRestDifference','timeDifferenceSinceChange','speedFromLastEvent',
                 'playerPositionEncoder']

# assert False
# make sure to implement data checks to make sure you're doing regression on "proper" data
# and won't get very wrong results

for col in features_list[:-1]:
    shots = shots.withColumn(col,shots[str(col)].cast(DecimalType()))

shots = shots.fillna('Aaa',subset=['playerPositionThatDidEvent'])
# use Aaa for null values and then order the values alphabetically so that the
# coefficients will show the differential of the player's position

shots = shots.withColumn('goal',shots.goal.cast(IntegerType()))

In [4]:
player_positions = shots.select('playerPositionThatDidEvent').distinct().collect()[:]
player_positions = [x[0] for x in player_positions]

player_indexer = StringIndexer(inputCol='playerPositionThatDidEvent',outputCol='playerPositionIndexer',
                               handleInvalid='skip',stringOrderType='alphabetAsc')
player_encoder = OneHotEncoderEstimator(inputCols=['playerPositionIndexer'],outputCols=['playerPositionEncoder'])

In [21]:
vector_assembler = VectorAssembler(inputCols=features_list,outputCol='features')
pipeline = Pipeline(stages=[player_indexer,player_encoder,vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training, test = piped_data.randomSplit([0.7,0.3])
lr = LogisticRegression(labelCol='goal')
evaluator = BinaryLogLossEvaluator(labelCol='goal')
grid = (ParamGridBuilder()
           .addGrid(lr.regParam,[0,0.001,0.01,0.1])
           .addGrid(lr.elasticNetParam,[0,0.5,1])
           .build())
cv = CrossValidator(estimator=lr,
                   estimatorParamMaps=grid,
                    evaluator=evaluator)

models = cv.fit(training)
cv_best_lr = models.bestModel
cv_best_lr.write().overwrite().save('models/cv_best_lr')

# cv_best_lr = LogisticRegressionModel.load('../models/cv_best_lr')

In [42]:
vector_assembler = VectorAssembler(inputCols=features_list,outputCol='features')
pipeline = Pipeline(stages=[player_indexer,player_encoder,vector_assembler])

piped_data = pipeline.fit(shots).transform(shots)
training, test = piped_data.randomSplit([0.7,0.3])
dtc = DecisionTreeClassifier(labelCol='goal')

best_dtc = dtc.fit(training)


In [57]:
predictions = best_dtc.transform(test)
evaluator = BinaryLogLossEvaluator(labelCol='goal')
evaluator.evaluate(predictions)

In [48]:
predictions = LogisticRegression(labelCol='goal').fit(training).transform(test)
BinaryLogLossEvaluator(labelCol='goal').evaluate(predictions)

82660.95925334083

In [53]:
predictions = RandomForestClassifier(labelCol='goal').fit(training).transform(test)
BinaryLogLossEvaluator(labelCol='goal').evaluate(predictions)

86275.56688397292

In [51]:
features_list

['arenaAdjustedShotDistance',
 'shotAngleAdjusted',
 'shotRush',
 'skaterDifference',
 'shotOnEmptyNet',
 'shotRebound',
 'offWing',
 'shooterTimeOnIce',
 'averageRestDifference',
 'timeDifferenceSinceChange',
 'speedFromLastEvent',
 'playerPositionEncoder']

In [44]:
predictions.columns

['shotID',
 'homeTeamCode',
 'awayTeamCode',
 'season',
 'isPlayoffGame',
 'game_id',
 'homeTeamWon',
 'id',
 'time',
 'timeUntilNextEvent',
 'timeSinceLastEvent',
 'period',
 'team',
 'location',
 'event',
 'goal',
 'shotPlayContinuedOutsideZone',
 'shotPlayContinuedInZone',
 'shotGoalieFroze',
 'shotPlayStopped',
 'shotGeneratedRebound',
 'homeTeamGoals',
 'awayTeamGoals',
 'xCord',
 'yCord',
 'xCordAdjusted',
 'yCordAdjusted',
 'shotAngle',
 'shotAngleAdjusted',
 'shotAnglePlusRebound',
 'shotAngleReboundRoyalRoad',
 'shotDistance',
 'shotType',
 'shotOnEmptyNet',
 'shotRebound',
 'shotAnglePlusReboundSpeed',
 'shotRush',
 'speedFromLastEvent',
 'lastEventxCord',
 'lastEventyCord',
 'distanceFromLastEvent',
 'lastEventShotAngle',
 'lastEventShotDistance',
 'lastEventCategory',
 'lastEventTeam',
 'homeEmptyNet',
 'awayEmptyNet',
 'homeSkatersOnIce',
 'awaySkatersOnIce',
 'awayPenalty1TimeLeft',
 'awayPenalty1Length',
 'homePenalty1TimeLeft',
 'homePenalty1Length',
 'playerPositionTha

In [9]:
cv_best_lr.extractParamMap()

{Param(parent='LogisticRegression_4fa4ec7f1bb1', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='maxIter', doc='maximum number of iterations (>= 0)'): 100,
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LogisticRegression_4fa4ec7f1bb1', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability esti

In [7]:
xG.select('probability').show(5)

+--------------------+
|         probability|
+--------------------+
|[0.95282700887131...|
|[0.97289266656632...|
|[0.97538017600326...|
|[0.96099172149719...|
|[0.96153512065908...|
+--------------------+
only showing top 5 rows



In [54]:
xG = best_lr.transform(piped_data)

def get_xGoal(row):
    return row[1].item()

udf_get_xGoal = func.udf(get_xGoal,DoubleType())

xG = xG.withColumn('xGoalPred',udf_get_xGoal(xG.select('probability')[0]))

In [55]:
result = xG.filter('skaterDifference=0 AND shootingTeamForwardsOnIce+shootingTeamDefencemenOnIce=5').groupBy('shooterName','shooterPlayerId','season').agg({'xGoalPred':'sum'}).sort('sum(xGoalPred)',ascending=False)

In [56]:
result.sort('sum(xGoalPred)',ascending=False).show(25)

+-----------------+---------------+------+------------------+
|      shooterName|shooterPlayerId|season|    sum(xGoalPred)|
+-----------------+---------------+------+------------------+
|    Alex Ovechkin|        8471214|  2008|35.783767478299744|
|    Alex Ovechkin|        8471214|  2007| 30.29776217049168|
|    Alex Ovechkin|        8471214|  2017|28.234854057280614|
|       Eric Staal|        8470595|  2008|27.915661981138122|
|Henrik Zetterberg|        8468083|  2007| 26.43356825682126|
|    Alex Ovechkin|        8471214|  2014|26.385579989517183|
|      Brent Burns|        8470613|  2015|25.160681705823734|
|    Alex Ovechkin|        8471214|  2009|24.411166033552515|
|     Marian Hossa|        8466148|  2008|24.196189135171608|
|    Alex Ovechkin|        8471214|  2010| 24.18904021550476|
|    Alex Ovechkin|        8471214|  2015|23.982680733302757|
|       Timo Meier|        8478414|  2018|23.841017600195162|
|      Brent Burns|        8470613|  2017|23.776273658737196|
|      Z

In [58]:
xG = best_lr.transform(piped_data)

def get_xGoal(row):
    return row[1].item()

udf_get_xGoal = func.udf(get_xGoal,DoubleType())

xG = xG.withColumn('xGoalPred',udf_get_xGoal(xG.select('probability')[0]))

In [59]:
result = xG.filter('skaterDifference=0 AND shootingTeamForwardsOnIce+shootingTeamDefencemenOnIce=5').groupBy('shooterName','shooterPlayerId','season').agg({'xGoalPred':'sum'}).sort('sum(xGoalPred)',ascending=False)

In [60]:
result.sort('sum(xGoalPred)',ascending=False).show(25)

+-----------------+---------------+------+------------------+
|      shooterName|shooterPlayerId|season|    sum(xGoalPred)|
+-----------------+---------------+------+------------------+
|    Alex Ovechkin|        8471214|  2008|35.783767478299744|
|    Alex Ovechkin|        8471214|  2007| 30.29776217049168|
|    Alex Ovechkin|        8471214|  2017|28.234854057280614|
|       Eric Staal|        8470595|  2008|27.915661981138122|
|Henrik Zetterberg|        8468083|  2007| 26.43356825682126|
|    Alex Ovechkin|        8471214|  2014|26.385579989517183|
|      Brent Burns|        8470613|  2015|25.160681705823734|
|    Alex Ovechkin|        8471214|  2009|24.411166033552515|
|     Marian Hossa|        8466148|  2008|24.196189135171608|
|    Alex Ovechkin|        8471214|  2010| 24.18904021550476|
|    Alex Ovechkin|        8471214|  2015|23.982680733302757|
|       Timo Meier|        8478414|  2018|23.841017600195162|
|      Brent Burns|        8470613|  2017|23.776273658737196|
|      Z

In [26]:
result.filter('shooterPlayerId=8471214').show()

+-------------+---------------+------+------------------+
|  shooterName|shooterPlayerId|season|    sum(xGoalPred)|
+-------------+---------------+------+------------------+
|Alex Ovechkin|        8471214|  2008|33.827255101227664|
|Alex Ovechkin|        8471214|  2017| 30.22094161889863|
|Alex Ovechkin|        8471214|  2007|28.852769501283888|
|Alex Ovechkin|        8471214|  2014|25.286749934223458|
|Alex Ovechkin|        8471214|  2015|24.227578888874504|
|Alex Ovechkin|        8471214|  2010|23.684240378194218|
|Alex Ovechkin|        8471214|  2009|22.688682775987218|
|Alex Ovechkin|        8471214|  2011| 21.90472505923295|
|Alex Ovechkin|        8471214|  2016|21.326990707115755|
|Alex Ovechkin|        8471214|  2018|19.774078894199647|
|Alex Ovechkin|        8471214|  2013|17.490679643872134|
|Alex Ovechkin|        8471214|  2012|14.351561710830326|
+-------------+---------------+------+------------------+

